# LINEAR ALGEBRA

# Contents
- Linear Algebra
    - Overview
    - Vectors
    - Matrices
    - Solving Systems of Equations
    - Eigenvalues and Eigenvectors
    - Further Topics
    - Exercises


## Overview
Linear algebra is one of the most useful branches of applied mathematics for economists to invest in.

For example, many applied problems in economics and finance require the solution of a linear system of equations, such as

$$
y_1 = ax_1+bx_2\\
y_2 = cx_1+dx_2\\
$$

or, more generally,

<a id='system-of-equations'></a>
$$
\begin{aligned}
y_1 = a_{11}x_1+a_{12}x_2+\dots+a_{1k}x_k\\
\vdots\\
y_n = a_{n1}x_1+a_{n2}x_2+\dots+a_{nk}x_k\\
\end{aligned}\tag{2.1}
$$

The objective here is to solve for the “unknowns” $x_1,\dots,x_k$ given $a_{11},\dots,a_{nk}$ and $y_1,\dots,y_n$. When considering such problems, it is essential that we first consider at least some of the following questions
- Does a solution actually exist?
- Are there in fact many solutions, and if so how should we interpret them?
- If no solution exists, is there a best “approximate” solution?
- If a solution exists, how should we compute it?

These are the kinds of topics addressed by linear algebra. We will cover the basics of linear and matrix algebra, treating both theory and computation.

Let’s start with some imports:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from scipy.linalg import inv, solve, det, eig

plt.rcParams["figure.figsize"] = (11, 5)  #set default figure size

## Vectors

A vector of length $n$ is just a sequence (or array, or tuple) of $n$ numbers, which we write as $x=(x_1,\dots,x_n)$ or $x=[x_1,\dots,x_n]$ 

We will write these sequences either horizontally or vertically as we please. (Later, when we wish to perform certain matrix operations, it will become necessary to distinguish between the two). The set of all $n$-vectors is denoted by $\R^n$